In [1]:
import sys
for i in sys.path:
    print(i)

C:\Users\nicky\Desktop\3-2학기\빅데이터분석
C:\Anaconda3\python38.zip
C:\Anaconda3\DLLs
C:\Anaconda3\lib
C:\Anaconda3

C:\Anaconda3\lib\site-packages
C:\Anaconda3\lib\site-packages\locket-0.2.1-py3.8.egg
C:\Anaconda3\lib\site-packages\win32
C:\Anaconda3\lib\site-packages\win32\lib
C:\Anaconda3\lib\site-packages\Pythonwin
C:\Anaconda3\lib\site-packages\IPython\extensions
C:\Users\nicky\.ipython


In [27]:
!where python
#!where pyspark


C:\Anaconda3\python.exe
C:\Users\nicky\AppData\Local\Programs\Python\Python39\python.exe
C:\Users\nicky\AppData\Local\Microsoft\WindowsApps\python.exe


In [15]:
import os
os.environ["PYSPARK_PYTHON"]="C:\Anaconda3\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"]="C:\Anaconda3\python.exe"

## SparkSession 생성
* Spark를 사용하려면 SparkSession 객체를 생성해야 한다.

SparkSession을 생성해 보자. SparkSesion은 sql 모듈로 'pyspark.sql.SparkSession'을 클라이언트로 사용한다. 필요한 설정은 SparkSession이 만들지기 전에 해 두어야 한다. 여기서는 설정을 별도로 하지 않고 비워 놓았다. SparkSession은 builder.getOrCreate() 함수를 호출하여, 기존의 session 또는 새로 생성하여 사용한다. 함수 getOrCreate() 함수는 singleton 패턴으로 한 번에 하나의 세션만이 존재하도록 한다. SparkSession을 종료하려면 stop() 함수를 호출한다.

spark = pyspark.sql.SparkSession.builder.getOrCreate()

spark.stop()

Spark를 실행하기 전 필수적으로 설정해야 하는 항목은

* master: (1) 분산의 경우 master URL 또는 (2) 로컬인 경우 local[]라고 적어준다. 즉 local의 수는 CPU core의 수를 의미한다. 예를 들어 local[*]는 가능한 최대한의 core를 사용한다는 의미이다. 예를 들어, local[5]라고 하면, core의 수가 2개라고 하더라도 데이터는 5개의 partitions로 나누어져 주어진다.
 * local은 Spark를 로컬에서 실행한다는 의미이다.
 * local[n]는 worker의 쓰레드를 n개로 한다는 의미. CPU core의 개수에 맞추어 설정하자.
 * local[*] 는 가능하면 가용한 모든 쓰레드를 사용한다는 의미 (Runtime.getRuntime.availableProcessors()로 그 수를 알 수 있다)
* appName: 앱의 이름

In [1]:
import pyspark

myConf=pyspark.SparkConf()
#myConf=pyspark.SparkConf().set("spark.driver.bindAddress", "127.0.0.1")
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [2]:
print ("Spark version \t: {}".format(spark.version))
print ("Spark App \t: {}".format(spark.conf.get('spark.app.name')))
print ("Spark Master \t: {}".format(spark.conf.get('spark.master')))
print ("Spark Host \t: {}".format(spark.conf.get('spark.driver.host')))

Spark version 	: 3.1.2
Spark App 	: myApp
Spark Master 	: local
Spark Host 	: LAPTOP-LKQ849JQ


In [3]:
spark.conf.get('spark.sql.warehouse.dir')

'file:/C:/Users/nicky/Desktop/3-2학기/빅데이터분석/spark-warehouse'

In [4]:
spark.sparkContext.getConf().getAll()

[('spark.master', 'local'),
 ('spark.sql.warehouse.dir',
  'file:/C:/Users/nicky/Desktop/3-2학기/빅데이터분석/spark-warehouse'),
 ('spark.app.name', 'myApp'),
 ('spark.app.startTime', '1632022678320'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.host', 'LAPTOP-LKQ849JQ'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.id', 'local-1632022680739'),
 ('spark.driver.port', '10649')]

In [5]:
spark.sparkContext._conf.getAll()

[('spark.master', 'local'),
 ('spark.sql.warehouse.dir',
  'file:/C:/Users/nicky/Desktop/3-2학기/빅데이터분석/spark-warehouse'),
 ('spark.app.name', 'myApp'),
 ('spark.app.startTime', '1632022678320'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.host', 'LAPTOP-LKQ849JQ'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.id', 'local-1632022680739'),
 ('spark.driver.port', '10649')]

 설정의 변경

SparkSession이 일단 만들어지고 나서는, sparkContext를 경유해서 

spark.sparkContext._conf.set() 함수로 설정을 변경할 수 있다.


Spark를 실행하면서 라이브러리를 설정해야 할 필요도 생겨나게 된다. 다음은 mongo, graphframes, csv 등의 라이브러리가 설정되어 있는 내용을 보여주고 있다. 라이브러리는 https://spark-packages.org 를 방문해서 찾아서 사용하면 된다. 실제로 설치하지 않고, 명명규칙에 따라, maven에서 하는 것과 같이, 콜론으로 구분해, 패키지명과 라이브러리를 버전정보와 같이 적어주면 된다.

In [6]:
myList=[1,2,3,4,5,6,7]

In [7]:
myRdd1 = spark.sparkContext.parallelize(myList)


In [10]:
myRdd1.collect()

[1, 2, 3, 4, 5, 6, 7]

In [ ]:
%%writefile data/ds_spark_wiki.txt
Wikipedia
Apache Spark is an open source cluster computing framework.
아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.
Apache Spark Apache Spark Apache Spark Apache Spark
아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크
Originally developed at the University of California, Berkeley's AMPLab,
the Spark codebase was later donated to the Apache Software Foundation,
which has maintained it since.
Spark provides an interface for programming entire clusters with
implicit data parallelism and fault-tolerance.

In [ ]:
import os
myRdd2=spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_wiki.txt"))

In [ ]:
myRdd2.first()

In [ ]:
import os
myDf=spark.read.text(os.path.join("data", "ds_spark_wiki.txt"))
print (myDf.first())

In [ ]:
print (type(myDf))

In [ ]:
%%writefile ./data/ds_spark_2cols.csv
35, 2
40, 27
12, 38
15, 31
21, 1
14, 19
46, 1
10, 34
28, 3
48, 1
16, 2
30, 3
32, 2
48, 1
31, 2
22, 1
12, 3
39, 29
19, 37
25, 2

In [ ]:
myRdd4 = spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_2cols.csv"))

In [ ]:
myList=myRdd4.take(5)
print (type(myList))

In [ ]:
spark